# Coursework 1

This notebook is intended to be used as a starting point for your experiments. The instructions can be found in the instructions file located under spec/coursework1.pdf. The methods provided here are just helper functions. If you want more complex graphs such as side by side comparisons of different experiments you should learn more about matplotlib and implement them. Before each experiment remember to re-initialize neural network weights and reset the data providers so you get a properly initialized experiment. For each experiment try to keep most hyperparameters the same except the one under investigation so you can understand what the effects of each are.

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
os.environ['MLP_DATA_DIR']="../data"

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

def slugify(value):
    """
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens.
    """
    import re
    value = re.sub(r'[^\w\s/\\.-]', '', value).strip().lower()
    value = re.sub(r'[-\s/\\]+', '-', value)
    return value


def train_model_and_plot_stats(
        model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True, title="Plot"):
    
    # As well as monitoring the error over training also monitor classification
    # accuracy i.e. proportion of most-probable predicted classes being equal to targets
    data_monitors={'acc': lambda y, t: (y.argmax(-1) == t.argmax(-1)).mean()}

    # Use the created objects to initialise a new Optimiser instance.
    optimiser = Optimiser(
        model, error, learning_rule, train_data, valid_data, data_monitors, notebook=notebook)

    # Run the optimiser for 5 epochs (full passes through the training set)
    # printing statistics every epoch.
    stats, keys, run_time = optimiser.train(num_epochs=num_epochs, stats_interval=stats_interval)

    # Plot the change in the validation and training set error over training.
    fig_1 = plt.figure(figsize=(8, 4))
    ax_1 = fig_1.add_subplot(111)
    for k in ['error(train)', 'error(valid)']:
        ax_1.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_1.legend(loc=0)
    ax_1.set_xlabel('Epoch number')
    fig_1.tight_layout()
    fig_1.savefig('figs/' + slugify(title) + "_error.pdf")

    # Plot the change in the validation and training set accuracy over training.
    fig_2 = plt.figure(figsize=(8, 4))
    ax_2 = fig_2.add_subplot(111)
    for k in ['acc(train)', 'acc(valid)']:
        ax_2.plot(np.arange(1, stats.shape[0]) * stats_interval, 
                  stats[1:, keys[k]], label=k)
    ax_2.legend(loc=0)
    ax_2.set_xlabel('Epoch number')
    fig_2.tight_layout()
    fig_2.savefig('figs/' + slugify(title) + "_acc.pdf")
    return stats, keys, run_time, fig_1, ax_1, fig_2, ax_2

In [3]:
# The below code will set up the data providers, random number
# generator and logger objects needed for training runs. As
# loading the data from file take a little while you generally
# will probably not want to reload the data providers on
# every training run. If you wish to reset their state you
# should instead use the .reset() method of the data providers.
import numpy as np
import logging
from mlp.data_providers import MNISTDataProvider, EMNISTDataProvider

# Seed a random number generator
seed = 11102018 
rng = np.random.RandomState(seed)
batch_size = 100
# Set up a logger object to print info about the training run to stdout
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.handlers = [logging.StreamHandler()]

# Create data provider objects for the MNIST data set
train_data = EMNISTDataProvider('train', batch_size=batch_size, rng=rng)
valid_data = EMNISTDataProvider('valid', batch_size=batch_size, rng=rng)

KeysView(<numpy.lib.npyio.NpzFile object at 0x00000165054FEBE0>)
KeysView(<numpy.lib.npyio.NpzFile object at 0x00000165054FE4E0>)


In [ ]:
# The model set up code below is provided as a starting point.
# You will probably want to add further code cells for the
# different experiments you run.

from mlp.layers import AffineLayer, SoftmaxLayer, SigmoidLayer, ReluLayer, LeakyReluLayer
from mlp.errors import CrossEntropySoftmaxError
from mlp.models import MultipleLayerModel
from mlp.initialisers import ConstantInit, GlorotUniformInit
from mlp.learning_rules import GradientDescentLearningRule
from mlp.optimisers import Optimiser

#setup hyperparameters
learning_rates = [0.1, 0.5, 1, 2]
input_dim, output_dim, hidden_dim = 784, 47, 100
stats_interval = 10
num_epochs = 200
n_hidden_layers = range(2, 6)
rule_to_test = GradientDescentLearningRule

for n_hidden_layer in n_hidden_layers:
    for learning_rate in learning_rates:
        title = f"{rule_to_test.__name__} num_epochs={num_epochs}, lr={learning_rate}, n_hidden={n_hidden_layer}"
        weights_init = GlorotUniformInit(rng=rng)
        biases_init = ConstantInit(0.)
        layers = [ # Hidden 1
            AffineLayer(input_dim, hidden_dim, weights_init, biases_init), 
            ReluLayer()
        ]
        for _ in range(n_hidden_layer - 1):
            layers.append(AffineLayer(hidden_dim, hidden_dim, weights_init, biases_init)) 
            layers.append(ReluLayer())
        layers.append(AffineLayer(hidden_dim, output_dim, weights_init, biases_init)) # Output
        model = MultipleLayerModel(layers)

        error = CrossEntropySoftmaxError()
        # Use a basic gradient descent learning rule
        learning_rule = rule_to_test()

        #Remember to use notebook=False when you write a script to be run in a terminal
        _ = train_model_and_plot_stats(
            model, error, learning_rule, train_data, valid_data, num_epochs, stats_interval, notebook=True, title=title)

Epoch 10: 4.6s to complete
    error(train)=2.28e+00, acc(train)=4.25e-01, error(valid)=2.27e+00, acc(valid)=4.23e-01


Epoch 20: 4.4s to complete
    error(train)=1.50e+00, acc(train)=5.86e-01, error(valid)=1.50e+00, acc(valid)=5.82e-01


Epoch 30: 4.3s to complete
    error(train)=1.28e+00, acc(train)=6.40e-01, error(valid)=1.29e+00, acc(valid)=6.32e-01


Epoch 40: 4.8s to complete
    error(train)=1.16e+00, acc(train)=6.73e-01, error(valid)=1.17e+00, acc(valid)=6.67e-01


Epoch 50: 4.5s to complete
    error(train)=1.07e+00, acc(train)=6.96e-01, error(valid)=1.08e+00, acc(valid)=6.91e-01


...

Epoch 20: 5.3s to complete
    error(train)=1.47e+00, acc(train)=5.94e-01, error(valid)=1.47e+00, acc(valid)=5.90e-01


Epoch 30: 4.1s to complete
    error(train)=1.27e+00, acc(train)=6.44e-01, error(valid)=1.27e+00, acc(valid)=6.39e-01


Epoch 40: 3.6s to complete
    error(train)=1.15e+00, acc(train)=6.74e-01, error(valid)=1.16e+00, acc(valid)=6.69e-01


Epoch 50: 4.3s to complete
    error(train)=1.06e+00, acc(train)=6.97e-01, error(valid)=1.07e+00, acc(valid)=6.90e-01


Epoch 60: 4.0s to complete
    error(train)=9.90e-01, acc(train)=7.16e-01, error(valid)=1.00e+00, acc(valid)=7.12e-01
